# Simulated growth in a test tube

# Calculations

In [2]:
# libraries
import numpy as np
from src.evap_standard_curve import *

<div class="alert alert-success">
    Modify your parameters here
</div>

In [3]:
# parameters

# ! neither v_s or v_d should > 10 ul since we have a p10 pipette
tau = 6    # hours. doubling time
v_s = 9    # ul. volume per sample
t_s = 4    # hours. time interval between samples
        # rehydration frequency is the same as sampling frequency
v_d = 3    # dump volume. the frequency of dumping is the same as sampling.
        # this is useful to avoid very small initial volume/growth volume
    
SAMPLE_OFFSET = 0    # use if not starting from A1
T_TOT = 72

# constants

V_MIN = 1    # ul. minimal volume the pipette can handle in one operation
T_MIN = 0.5    # hours. minimal time interval between contiguous steps in a sequence, e.g., two  rehydration steps

In [4]:
# find out the "balancing" initial volume

# v_removed = v_b * ln(2) * t_s / tau
# where v_b is the balancing volume

v_removed = v_s + v_d    # in time step t_s, how much volume is removed
v_b = v_removed * tau / (np.log(2) * t_s)
v_b = np.round(v_b, decimals=1)    # restricted by precision of the pipette

print("Initial volume should be", v_b, "ul")

Initial volume should be 26.0 ul


In [5]:
# find the "growth" interval
# notice that the strategy is only a function of sampling vol and interval, and discard vol, but not tau
# (tau determines initial vol)

v_removed = v_s + v_d    # which is also the total growth vol

max_n_pip = int(np.floor(t_s / T_MIN))    # max number of pipetting steps, restricted by T_MIN

v_growth = -1    # flag for no strategy
# starting from the max # pipettes
for n_pip in range(max_n_pip + 1, 1, -1):
    
    # i must be a factor of max__pip
    if max_n_pip % n_pip != 0:
        continue
        
    v_cur = np.round(v_removed / n_pip, decimals=1)
    #print("Trying 'growth' volume", v_cur, "ul, pipetting", n_pip, "times per sampling...")
    
    if v_cur >= 3 * V_MIN and v_cur <= 10:
        # the smallest v_growth satisfies min. pipetting precision is what we want
        # v_growth consists of 3 pipetting steps, thus x3
        v_growth = v_cur
        break

if v_growth == -1:
    print("No 'growth' volume found!")
else:
    print("Given that", v_removed, "ul is removed per", t_s, "hours",
          ", the total volume should 'grow' by", v_growth, "ul for", n_pip, "times (interval =",
          t_s/n_pip, "hours) in the same period.")

Given that 12 ul is removed per 4 hours , the total volume should 'grow' by 3.0 ul for 4 times (interval = 1.0 hours) in the same period.


In [6]:
# find the rehydration vol

if t_s == 2:
    v_hydrate = evap_vol_2h(v_s / 3 * 4)
elif t_s == 4:
    v_hydrate = evap_vol_4h(v_s / 3 * 4)
else:
    raise Exception("Rehydration interval can be only either 2 or 4 hours")

print("Rehydration vol each time is", v_hydrate)

Rehydration vol each time is 4.1


# Initialization

In [7]:
# current working dir is /var/lib/jupyter/notebooks
import numpy as np
import opentrons.execute
from opentrons import protocol_api
from src.pipette_viscous import transfer_viscous,aspirate_viscous, dispense_viscous
import src.scheduler as scheduler
from src.evap_standard_curve import *

# start the protocol context
protocol = opentrons.execute.get_protocol_api("2.11")

# home is required
protocol.set_rail_lights(False)
protocol.home()

Out of bounds move: X=(418.00000000000006 motor controller, 416.9673140124148 deck) too high for limit 418.0
Out of bounds move: X=(418.00000000000006 motor controller, 416.9673140124148 deck) too high for limit 418.0


## Load labware

- The incubator is on slot 11
- The alluminum rack is mounted to the incubator
- The GEB tiprack is on slot 2, 5, 6, 8, 9
- The Bio-rad PCR plate is on slot 3
- The P10 1st gen pipette is on the left

In [8]:
incubator = protocol.load_module("temperature module", 11)

rack = incubator.load_labware("opentrons_24_aluminumblock_fisher_1.5ml_centrifuge")    # the rack is mounted upon the temp. module (don't specify slot)

tip_rack_1 = protocol.load_labware("geb_taller_96_tiprack_10ul", '2')    # will consume 380 tips in total, about 4 boxes
tip_rack_2 = protocol.load_labware("geb_taller_96_tiprack_10ul", '5')
tip_rack_3 = protocol.load_labware("geb_taller_96_tiprack_10ul", '6')
tip_rack_4 = protocol.load_labware("geb_taller_96_tiprack_10ul", '8')
tip_rack_5 = protocol.load_labware("geb_taller_96_tiprack_10ul", '9')    # custom labware. see definition at labware/

plate = protocol.load_labware("biorad_96_wellplate_200ul_pcr", '3')

pipette = protocol.load_instrument("p10_single", "left", 
                                   tip_racks = [tip_rack_1, tip_rack_2, tip_rack_3, tip_rack_4, tip_rack_5])

# Define liquid/tubes

In [9]:
RXN_TUBE = rack.wells_by_name()["A1"]
DYE_TUBE = rack.wells_by_name()["A3"]
WAT_TUBE = rack.wells_by_name()["A5"]
DIS_TUBE = rack.wells_by_name()["C1"]    # discard
A_TUBE = rack.wells_by_name()["C3"]    # 3x KaiA
B_TUBE = rack.wells_by_name()["C4"]    # 3x KaiB
U_TUBE = rack.wells_by_name()["C5"]    # 3x unphosphorylated KaiC

# Define operations

In [13]:
def sample_and_discard(idx):
    "take a sample from the tube to a 96-well plate"

    # current well
    cur_well = plate.wells()[idx + SAMPLE_OFFSET]

    # pipette the dye
    transfer_viscous(pipette, protocol, v_s / 3, DYE_TUBE, cur_well)

    # pipette the sample
    transfer_viscous(pipette, protocol, v_s, RXN_TUBE, cur_well, if_mix=True)
    
    # discard from reaction
    aspirate_viscous(pipette, protocol, v_d, RXN_TUBE)
    pipette.dispense(v_d, DIS_TUBE)
    pipette.drop_tip()

# for logging
str_sample_and_discard = "Sample and mix with loading buffer, and discard from the reaction"

def rehydrate(idx):
    "to prevent drying down"

    # determine rehydration vol
    if t_s == 2:
        v_hydrate = evap_vol_2h(v_s / 3 * 4)
    elif t_s == 4:
        v_hydrate = evap_vol_4h(v_s / 3 * 4)
    else:
        raise Exception("Rehydration interval can be only either 2 or 4 hours")
    
    for i in range(idx):
        cur_well = plate.wells()[i + SAMPLE_OFFSET]
        
        pipette.pick_up_tip()
        pipette.aspirate(v_hydrate, WAT_TUBE.bottom())    # just water, no oil
        dispense_viscous(pipette, protocol, v_hydrate, cur_well, if_mix=True)
        
str_rehydrate = "Rehydrate to prevent drying down"
        
def grow(v_g):
    "dilute the reaction with U-KaiC, KaiA, and KaiB by volume v_g"
    
    v_g_per_tube = np.round(v_g / 3, 1)
    
    for from_tube in [U_TUBE, A_TUBE, B_TUBE]:
        aspirate_viscous(pipette, protocol, v_g_per_tube, from_tube)
        # be sure to dispense at the bottom
        dispense_viscous(pipette, protocol, v_g_per_tube, RXN_TUBE, disp_height=0, if_mix=True, if_blowout=False)
        # it's also okay to mix KaiA and KaiB in one tube, then all conc are 2x
    
str_grow = "Simulate growth"

# Define instructions

## This is a test for each operation

In [7]:
# scheduler.drop()
# scheduler.cat(time_vec=[0, 0, 0, 0],
#              func_vec=[sample_and_discard, sample_and_discard, rehydrate, grow],
#              param_vec=[(0,), (1,), (2,), (4.5,)],
#              str_vec=[str_sample_and_discard, str_sample_and_discard, str_rehydrate, str_grow],
#              n_tip_vec=[3, 3, 2, 3])
# scheduler.report(unit="hours")

A total of 9 tips is required

At 0.00 hours, Sample and mix with loading buffer, and discard from the reaction, with params (0,)
At 0.00 hours, Sample and mix with loading buffer, and discard from the reaction, with params (1,)
At 0.00 hours, Rehydrate to prevent drying down, with params (2,)
At 0.00 hours, Simulate growth, with params (4.5,)


## The real instructions

In [11]:
incubator.set_temperature(30)

In [16]:
scheduler.drop()

# growth
t_growth = t_s / n_pip
n_steps_growth = int(T_TOT / t_growth)

scheduler.cat(time_vec=np.arange(1, n_steps_growth + 1) * t_growth * 60,
             func_vec=[grow] * n_steps_growth,
             param_vec=[(v_growth,)] * n_steps_growth,    # dilution volumes are all the same
             str_vec=[str_grow] * n_steps_growth,
             n_tip_vec=[3] * n_steps_growth)

# sampling
n_steps_s = int(T_TOT / t_s) + 1    # plus time 0

scheduler.cat(time_vec=np.arange(n_steps_s) * t_s * 60,
             func_vec=[sample_and_discard] * n_steps_s,
             param_vec=[ (i,) for i in range(n_steps_s) ],
             str_vec=[str_sample_and_discard] * n_steps_s,
             n_tip_vec=[3] * n_steps_s)

# rehydration
# scheduler.cat(time_vec=np.arange(1, n_steps_s) * t_s * 60,
#              func_vec=[rehydrate] * (n_steps_s - 1),
#              param_vec=[ (i,) for i in range(1, n_steps_s) ],
#              str_vec=[str_rehydrate] * (n_steps_s - 1),
#              n_tip_vec=[ i for i in range(1, n_steps_s) ])

scheduler.report(unit="hours")

A total of 273 tips is required

At 1.00 hours, Simulate growth, with params (3.0,)
At 2.00 hours, Simulate growth, with params (3.0,)
At 3.00 hours, Simulate growth, with params (3.0,)
At 4.00 hours, Simulate growth, with params (3.0,)
At 5.00 hours, Simulate growth, with params (3.0,)
At 6.00 hours, Simulate growth, with params (3.0,)
At 7.00 hours, Simulate growth, with params (3.0,)
At 8.00 hours, Simulate growth, with params (3.0,)
At 9.00 hours, Simulate growth, with params (3.0,)
At 10.00 hours, Simulate growth, with params (3.0,)
At 11.00 hours, Simulate growth, with params (3.0,)
At 12.00 hours, Simulate growth, with params (3.0,)
At 13.00 hours, Simulate growth, with params (3.0,)
At 14.00 hours, Simulate growth, with params (3.0,)
At 15.00 hours, Simulate growth, with params (3.0,)
At 16.00 hours, Simulate growth, with params (3.0,)
At 17.00 hours, Simulate growth, with params (3.0,)
At 18.00 hours, Simulate growth, with params (3.0,)
At 19.00 hours, Simulate growth, with pa

<div class="alert alert-success">
    Prepare the following liquid
</div>

In [15]:
V_SAFE = 50

v_rxn = v_b + v_removed    # time 0 sample
v_dye = v_s / 3 * n_steps_s + V_SAFE
v_wat = v_hydrate * sum(range(1, n_steps_s)) + V_SAFE
# actually water tube is 1 ml, without oil, and refilled every day
# water costs nothing...
v_stock = v_growth / 3 * n_steps_growth + V_SAFE

print(f"Reaction tube at A1, {v_rxn:.1f} ul")
print(f"Dye tube at A3, {v_dye:.1f} ul")
print(f"Water tube at A5, {v_wat:.1f} ul")
print(f"Stock protein tube at C3, C4, and C5, {v_stock:.1f} ul")

Reaction tube at A1, 38.0 ul
Dye tube at A3, 107.0 ul
Water tube at A5, 751.1 ul
Stock protein tube at C3, C4, and C5, 122.0 ul


<div class="alert alert-warning">
    <h2>STOP!</h2>
    Before you proceed, please check:<br />
    Have you placed the required labware?<br />
    Have you put the right volume of liquid in required position?
</div>

In [14]:
log_fn = "log/20220819_simulated_growth.log"
scheduler.run(protocol, log_fn)

Out of bounds move: X=(418.00000000000006 motor controller, 416.9673140124148 deck) too high for limit 418.0
alarm/error outside hard halt: ALARM: Hard limit +Z
alarm/error: command=M907 A0.1 B0.05 C0.05 X0.3 Y0.3 Z0.8 G4 P0.005 G0 Z150.17 

, resp=ALARM: Hard limit +Z
Move failed
Traceback (most recent call last):
  File "usr/lib/python3.7/site-packages/opentrons/drivers/smoothie_drivers/driver_3_0.py", line 900, in _send_command_unsynchronized
  File "usr/lib/python3.7/site-packages/opentrons/drivers/asyncio/communication/serial_connection.py", line 119, in send_command
  File "usr/lib/python3.7/site-packages/opentrons/drivers/asyncio/communication/serial_connection.py", line 140, in send_data
  File "usr/lib/python3.7/site-packages/opentrons/drivers/asyncio/communication/serial_connection.py", line 169, in _send_data
  File "usr/lib/python3.7/site-packages/opentrons/drivers/asyncio/communication/serial_connection.py", line 212, in raise_on_error
opentrons.drivers.asyncio.communicati

SmoothieError: SmoothieError: M907 A0.1 B0.05 C0.05 X0.3 Y0.3 Z0.8 G4 P0.005 G0 Z150.17 

 returned ALARM: Hard limit +Z